<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/TF_IDF_UP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])
stopwords.extend(['podemosen', 'podemen', 'ministro', 'ministra', 'ss', 'ruego', 'aprobado', 'aprobada'])
stopwords.extend(['diputado', 'diputada', 'don', 'doña', 'vicepresidenta', 'sagastizab', 'unzetabarrenetxea', 'agirretxea', 'urresti', 'escrutinio', 'dijo', 'dice', 'quevedo'])
stopwords.extend(['campuzano', 'beitialarrangoitia', 'gonzalezmoro', 'mixto'])

In [5]:
# Se incorporan los nombres de los oradores como stopwords
from collections import Counter
nombres=[]
for i in base.namey.unique().tolist(): nombres.extend(str(i).split(' '))
nombres = [x.strip(' ') for x in nombres]
nombres = [x.strip(' †') for x in nombres]
nombres = [x.strip(',') for x in nombres]
nombres=list(Counter(nombres))
nombres.remove('')

In [6]:
stopwords.extend(nombres)

In [7]:
partidos=pd.read_csv('/content/drive/MyDrive/partidos.csv', encoding='latin1', sep=';')

In [8]:
for x in partidos['nombre'].unique(): 
  a=str(x).rstrip(' ')
  stopwords.extend(str(a).split())
  stopwords.extend(a)

for x in partidos['1'].unique():
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

for x in partidos['2'].unique(): 
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

for x in partidos['3'].unique(): 
  a=str(x).rstrip(' ')
  stopwords.extend(a.split())
  stopwords.extend(a)

In [9]:
len(stopwords)

4481

In [10]:
# elimino acentos de los tokens
a,b = 'áàéèíóúü','aaeeiouu'
trans = str.maketrans(a,b)
list = []
for row in base.values:
    row[20]=row[20].translate(trans)
    list.append(row)

base = pd.DataFrame(list, columns=base.columns)
     

In [11]:
# llevo las stopwords a minúsculas y elimino acentos
stopwords=[x.lower() for x in stopwords if type(x)==str]
stopwords=[x.translate(trans) for x in stopwords if type(x)==str]

# TF-IDF

In [12]:
# El hiperparámetro min_df del TF-IDF elimina las palabras con menor frecuencia. Por ejemplo, las palabras que aparecen en menos que el 75% de los discursos. 
# A continuación se adopta un enfoque distinto: Se genera un vocabulario con todos los bigramas, y luego se elimina el 95% de los bigramas con menor TF-IDF (se conserva el 5% superior). 
# Es decir que entre los 8 millones de bigramas se seleccionan los 400mil de mayor TF-IDF. 

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ahuja', 'alcañiz', 'amor', 'arreciado', 'barcena', 'barnuevo', 'capel', 'capitan', 'cascos', 'castejon', 'cienfuegos', 'compromis', 'congr', 'escandon', 'estefani', 'figueroa', 'fortun', 'hickman', 'huarte', 'jeltzalea', 'jovellanos', 'lasquetty', 'latorre', 'legaz', 'lersundi', 'llin', 'machuca', 'maillo', 'mancisidor', 'margallo', 'maribona', 'marlaska', 'medel', 'menchero', 'mendicoa', 'na', 'olivares', 'patxi', 'peces', 'pedrero', 'pinilla', 'piñero', 'ponga', 'pujalte', 'salmones', 'santesmases', 'sito', 'soro', 'sotelo', 'tagle', 'tesorero', 'tizon', 'toledano', 'torrecilla', 'trevijano', 'valdecasas', 'valdes', 'vargas', 'villaseñor', 'ximo', 'yañez'] not in stop_words.
  warnings.warn(


In [13]:
total_features

7733805

In [14]:
sum_words = bow.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1])
vocabulary, _ = zip(*words_freq[:int(total_features * 0.95)])
less_vocabulary = vocabulary

In [15]:
voc=vect.vocabulary_.keys()-less_vocabulary

In [16]:
len(voc)

386691

In [17]:
cv=TfidfVectorizer(ngram_range=[2,2], vocabulary=voc , stop_words=stopwords)
vec = cv.fit(base['tokens'])

In [18]:
matrix = vec.transform(base['tokens'])

In [19]:
matrix.shape

(334421, 386691)

# LDA con 10 clusters

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

In [21]:
# Acá hay un tema porque se sigue usando 10 clusters cuando en realidad habría que chequear el número óptimo de clusters una vez definida la cantidad de bigramas a utilizar. 
# Sin embargo, "Text as data" dice que el número de clústers es generalmente arbitrario, y recomienda arrancar probando con 10. 
lda = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)     

In [22]:
lda.fit(matrix)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, random_state=0)

In [23]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx + 1))
        print(" , ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [24]:
display_topics(lda, feature_names=cv.get_feature_names_out(), no_top_words=20)

Topic 1:
numero formula , desean fijar , interpelacion urgent , rechazado numero , tiempo concluido , momento guarden , convalidacion derogacion , dirigida presidencia , concluir debat , industria energia , olabarria anchuelo , pregunto acepta , dirigida industria , guarden continu , formula lui , orama mismo , dirigida agricultura , transaccion pleno , acepta presentada , reanuda dirigida
Topic 2:
vaya concluyendo , acabado tiempo , legarda uriart , tarda coma , acatar constitucion , seccion relativo , fijar nombr , contestar interpelacion , ciuro buldo , azpiazu anchuelo , acepta alguna , capella farre , iglesia turrion , sorli fresquet , fijacion azpiazu , azpiazu gorriaran , pasamo interpelacion , eritja ciuro , mardon nombr , siento acabado
Topic 3:
extraordinaria comision , celebracion extraordinaria , acuerd celebracion , plural coscubiela , gorriaran plural , suficient miembro , permanent pertenecient , plural centella , resto plural , ocupen escaño , formulada plural , pertene

# LASSO por legislatura

In [25]:
from tabulate import tabulate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
from numpy import argsort
import statistics
from statistics import mode
from scipy.special import logsumexp
import scipy as sp

In [26]:
def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

In [27]:
frex_df=pd.DataFrame(frex(lda))

In [28]:
# Topic que presenta mayor FREX para cada bigrama
d = []
feature_names=vec.get_feature_names_out()
for n in enumerate(feature_names):
    d.append(
        {
            'N': n[0],
            'Bigrama': n[1],
            'Topic':  frex_df[n[0]].argsort()[9]
        }
    )

bi_topic_frex=pd.DataFrame(d)
bi_topic_frex.head(20)

,N,Bigrama,Topic
0,0,aabstencion nulo,5
1,1,aaizpun justicla,4
2,2,aarnau bandre,0
3,3,aazcarraga anasagasti,3
4,4,ab initio,8
5,5,ab intestato,8
6,6,ab soluto,7
7,7,ab tencion,8
8,8,abajo arriba,6
9,9,abajo firmant,6


In [30]:
# Relevancia de bigramas en la predicción por legislatura
tabla=[]
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
n=5

cv=TfidfVectorizer(ngram_range=[2,2], vocabulary=voc , stop_words=stopwords)

for x in legislaturas: 
  base_year=base[base.legislatura==x]
  
  vec = cv.fit(base_year['tokens'])
  train_matrix = vec.fit_transform(base_year['tokens']) 
  lr = LogisticRegression(penalty='l1', max_iter=50, solver='liblinear')
  lr.fit(train_matrix,base_year['party'])
  
  features=[]
  s=lr.coef_[0]
  
  print(f"Legislatura: {x} - Most Frecuent Topic: {mode(bi_topic_frex['Topic'][argsort(s)])}")
  print(tabulate([cv.get_feature_names_out()[argsort(s)[:n]], sorted(s)[:n],bi_topic_frex['Topic'][argsort(s)[:n]],cv.get_feature_names_out()[argsort(s)[-n:]], sorted(s)[-n:],  bi_topic_frex['Topic'][argsort(s)[-n:]]], tablefmt='pretty'))
     

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ahuja', 'alcañiz', 'amor', 'arreciado', 'barcena', 'barnuevo', 'capel', 'capitan', 'cascos', 'castejon', 'cienfuegos', 'compromis', 'congr', 'escandon', 'estefani', 'figueroa', 'fortun', 'hickman', 'huarte', 'jeltzalea', 'jovellanos', 'lasquetty', 'latorre', 'legaz', 'lersundi', 'llin', 'machuca', 'maillo', 'mancisidor', 'margallo', 'maribona', 'marlaska', 'medel', 'menchero', 'mendicoa', 'na', 'olivares', 'patxi', 'peces', 'pedrero', 'pinilla', 'piñero', 'ponga', 'pujalte', 'salmones', 'santesmases', 'sito', 'soro', 'sotelo', 'tagle', 'tesorero', 'tizon', 'toledano', 'torrecilla', 'trevijano', 'valdecasas', 'valdes', 'vargas', 'villaseñor', 'ximo', 'yañez'] not in stop_words.
  warnings.warn(


Legislatura: I - Most Frecuent Topic: 6
+---------------------+--------------------+--------------------+-------------------------+---------------------+
|    duda ninguna     | publicada boletin  |   articulo pausa   | interpelacion formulada |   nombr centrista   |
| -14.106612926414567 | -7.130562443810495 | -6.504855941926496 |   -5.572986201682329    | -5.156964089972759  |
|          6          |         6          |         5          |            6            |          8          |
|    pasamo examen    |   alguna manera    |   cualquier caso   |   ministerio hacienda   | presupuesto general |
| 4.6279455079329415  | 6.534625222880626  | 6.895209231519178  |    7.860193351317102    |  8.916653826721907  |
|          4          |         6          |         6          |            6            |          6          |
+---------------------+--------------------+--------------------+-------------------------+---------------------+
Legislatura: II - Most Frecuent Topic: 6
+------

# LASSO por año

In [31]:
# Relevancia de bigramas en la predicción por año
tabla=[]
vec = cv.fit(base['tokens'])
nombres=cv.get_feature_names_out()
componentes=lda.components_.transpose()
n = 5

for x in range (1979, 2010): 
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  train_matrix = vec.fit_transform(base_year['tokens'])
  y_train= base_year['party']
  lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
  lr.fit(train_matrix,y_train)

  s = lr.coef_[0]
    
  print(f"Año: {x}")
  print(tabulate([cv.get_feature_names_out()[argsort(s)[:n]], sorted(s)[:n],bi_topic_frex['Topic'][argsort(s)[:n]],cv.get_feature_names_out()[argsort(s)[-n:]], sorted(s)[-n:],  bi_topic_frex['Topic'][argsort(s)[-n:]]], tablefmt='pretty'))

Año: 1979
+---------------------+--------------------+---------------------+---------------------+---------------------+
|  publicada boletin  |  correspond ahora  |   general numero    |      quier uso      |   nombr industria   |
| -4.746888480813331  | -4.217645509783282 | -2.3159525201424147 | -2.0425211576243627 | -1.6908284433910525 |
|          6          |         6          |          6          |          9          |          0          |
| estatuto trabajador | comunidad autonoma |   comite empresa    | formacion politica  | convenio colectivo  |
| 0.6407872119927789  | 0.7547707843935254 | 1.5216464961861553  | 1.6893859469328585  | 1.7516272851437156  |
|          6          |         6          |          6          |          6          |          6          |
+---------------------+--------------------+---------------------+---------------------+---------------------+
Año: 1980
+--------------------+----------------------+---------------------+--------------------+----